(for local) \
CUDA Toolkit 11.8 \
cuDNN 8.9.x \
pip install torch==2.0.1 --extra-index-url https://download.pytorch.org/whl/cu118  
其餘套件自己想辦法

# Importing modules

In [1]:


import numpy as np
import math


import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader, TensorDataset, Subset

import csv

import multiprocessing as mp
import os
import sys
import copy
import random
import gc
import time
from tqdm import tqdm
from collections import defaultdict

import itertools

import dill

import warnings
warnings.filterwarnings('ignore')

from datasets import load_dataset

import torch
from transformers import BertTokenizer, BertModel
import numpy as np

from IPython.display import display, clear_output

In [2]:
from model import *

# Checking cuda

In [3]:
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    device_index = 0
    device = torch.device(f"cuda:{device_index}")
    print('using cuda...')
else:
    device = torch.device("cpu")
    print('using cpu...')

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

Device 0: NVIDIA GeForce RTX 4090
using cuda...


# Config

In [4]:
source = "squad" # squad  natural_questions

retrain = True

interval   = 100
train_size = 2000


In [5]:


max_length = 200

sequence_size =  max_length             
feature_size = 768          
num_layers = 3                      
num_heads = 4                
hidden_activation = 'tanh'
output_activation = 'tanh'
initializer = "xavier_normal"
optimizer = 'adam'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.000001       

num_epochs = 100  
batch_size = 1

model_directory = f'model.pth'


# Build model

In [6]:



# Initialize the model
model = build_model(sequence_size,
                    feature_size,
                    num_layers,
                    num_heads,
                    hidden_activation,
                    output_activation,
                    initializer,
                    optimizer,
                    loss,
                    bias,
                    drop_rate,
                    alpha)


model = model.to(device)

if retrain:
    model_dict = torch.load(model_directory)
    model.load_state_dict(model_dict[f'model'])


# Preparing data

In [ ]:
# Load the IMDB dataset
dataset     = load_dataset(source, trust_remote_code=True )
questions   = dataset['train']['question']
answers     = dataset['train']['answers' ]

# Ensure you don't pick more samples than available
train_size  = min(train_size, len(questions))

# 開始抽取資料
for _ in range(interval):

    # Randomly select indices
    random_indices = random.sample(range(len(questions)), train_size)

    # Get the selected subset of questions and answers
    questions = [questions[i] for i in random_indices]
    answers   = [answers[i]   for i in random_indices]

    # Create QA pairs
    qa_pairs  = []
    for question, answer in zip(questions, answers):
        qa_pairs.append(f"[CLS] {question} [SEP] {answer['text'][0]} [SEP] ")

    # Show the first 3 question-answer pairs
    for qa in qa_pairs[:3]:
        print(qa)
        print("-" * 50)

    # 初始化 BERT tokenizer 和 vectorizer
    tokenizer  = BertTokenizer.from_pretrained('bert-base-uncased')
    vectorizer = BertModel.from_pretrained('bert-base-uncased')

    # 範例句子
    # sentences = [
    #     "Hello, how are you? I am fine",
    #     "BERT is a powerful model.  I am fine",
    #     "Let's use transformers for NLP.  I am fine"
    #     "You are really a muta flicker.  I am fine"
    # ]
    sentences = qa_pairs

    # Step 1: Tokenize the sentences
    tokenized_sentences = tokenizer(sentences, add_special_tokens=False, padding='max_length', max_length=max_length, truncation=True, return_tensors="pt")

    # Step 2: Vectorize the sentences
    input_ids           = tokenized_sentences['input_ids']
    attention_masks     = tokenized_sentences['attention_mask']
    with torch.no_grad(): 
        input_vectors   = vectorizer(input_ids).last_hidden_state * ( attention_masks.unsqueeze(2) )

    def create_dataset(input_vectors, attention_masks):

        final_input  = []
        final_label  = []
        final_mask_1 = []
        final_mask_2 = []

        for i in range(input_vectors.size(0)): 
            for j in range(input_vectors.size(1) - 1):
                
                if attention_masks[i][j + 1] != 0:

                    factored_mask       = torch.zeros_like(attention_masks[i])
                    factored_mask[:j+1] = 1
                    input               = input_vectors[i] * factored_mask.unsqueeze(1)
                    final_input.append(input)

                    label = input_vectors[i][j+1]
                    final_label.append(label)

                    mask_2 = factored_mask.unsqueeze(1) * factored_mask.unsqueeze(0)
                    mask_2 = mask_2.unsqueeze(0)
                    mask_1 = (mask_2 -1) * 1e20
                    final_mask_1.append(mask_1)
                    final_mask_2.append(mask_2)

        final_input  = torch.stack(final_input , dim=0)
        final_label  = torch.stack(final_label , dim=0)
        final_mask_1 = torch.stack(final_mask_1, dim=0)
        final_mask_2 = torch.stack(final_mask_2, dim=0)

        return final_input, final_label, final_mask_1, final_mask_2

    # 生成資料集
    final_input, final_label, final_mask_1, final_mask_2 = create_dataset(input_vectors, attention_masks)

    # 彙整資料
    dataset    = TensorDataset(final_input, final_label, final_mask_1, final_mask_2)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


    # 訓練模型
    for epoch in range(num_epochs):
        
        model.train()  
        
        running_loss = 0.0

        for batch_idx, (input, label, mask_1, mask_2) in tqdm(enumerate(dataloader), total=len(dataloader), desc="Training Progress", ncols=100, unit="batch"):

            input  = input.to(device)
            label  = label.to(device)
            mask_1 = mask_1.to(device)
            mask_2 = mask_2.to(device)
            
            optimizer = model.optimizer
            optimizer.zero_grad()

            loss_function   = model.loss_function
            output          = model(input, (mask_1, mask_2))
            loss            = loss_function(output, label)
            loss.backward()     # get grad

            optimizer.step()    # update params

            running_loss += loss.item()
            
        epoch_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}] finished. Average Loss: {epoch_loss:.4f}")

        model_dict = {}
        model_dict[f'model'] = model.state_dict()
        torch.save(model_dict, model_directory)
        

In [ ]:

model_dict = {}
model_dict[f'model'] = model.state_dict()
torch.save(model_dict, model_directory)